# Behavioral Cloning

In [ ]:
import tensorflow as tf
import keras
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sklearn.utils
import sklearn.model_selection
import cv2
%matplotlib inline
print('Tensorflow version {0}'.format(tf.__version__))
print('Keras version {0}'.format(keras.__version__))

## File I/O
Want to read in a directory of jpg files and write out hdf5

In [ ]:
log = pd.read_csv('./driving_data/driving_log.csv', names=['CenterImg', 'LeftImg', 'RightImg', 
                                                           'SteeringAngle', 'Throttle', 'Break', 'Speed'])
log.head()

In [ ]:
log.describe()

### L-R Camera compensation

A correction factor of 0.1 degrees is added to the left camera and subtracted from the right camera to compensate for the lateral position of the camera relative to the center. The value of the correction factor should be evaluated.

In [ ]:
def change_brightness(img, lower_bound=0, upper_bound=255):
    '''
    Uniformly changes the brightness of an image randomly without overflowing.
    Change in brightness limited by original brightness of image
    '''
    img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    img[:,:,2] = img[:,:,2]/2 + np.random.randint(lower_bound, upper_bound)/2
    img = cv2.cvtColor(img, cv2.COLOR_HSV2RGB)
    return img

In [ ]:
images = []
measurements = []
image_path = []
offset = 0.15
new_scale = (256, 128)

for i in range(log.shape[0]):
    #Original    
    center = mpimg.imread(log['CenterImg'][i])
    left = mpimg.imread(log['LeftImg'][i])
    right = mpimg.imread(log['RightImg'][i])
    
    #Scale down images
#     center = cv2.resize(center, new_scale, interpolation=cv2.INTER_AREA)
#     left = cv2.resize(left, new_scale, interpolation=cv2.INTER_AREA)
#     right = cv2.resize(right, new_scale, interpolation=cv2.INTER_AREA)
    #Change brightness of images
    center = change_brightness(center)
    left = change_brightness(left)
    right = change_brightness(right)
    
    images.append(center)
    images.append(left)
    images.append(right)
    measurements.append(log['SteeringAngle'][i])
    measurements.append(log['SteeringAngle'][i]+offset)
    measurements.append(log['SteeringAngle'][i]-offset)
#     image_path.append(log['CenterImg'][i])
#     image_path.append(log['LeftImg'][i])
#     image_path.append(log['RightImg'][i])

    #Flipped
    images.append(np.fliplr(center))
    images.append(np.fliplr(left))
    images.append(np.fliplr(right))
    measurements.append(-log['SteeringAngle'][i])
    measurements.append(-log['SteeringAngle'][i]-offset)
    measurements.append(-log['SteeringAngle'][i]+offset)

### Unbalanced Classes
We would like to have an equal distribution of steering angles to reduce the bias for zero driving angle. We will apply sample weights to balance the effect of each class.

In [ ]:
# def bin_angles(y, n_bins = 20):
#     '''Computes a bin number for each steering angle in order to balance classes'''
#     return np.floor((y - np.min(y))*n_bins/(np.max(y) - np.min(y)))

def bin_weights(y, n_bins=10):
    '''Balances numbered classes and returns array of weights to apply to each class'''
    y_bins = np.floor((y - np.min(y))*n_bins/(np.max(y) - np.min(y)))
    class_weight = sklearn.utils.class_weight.compute_class_weight('balanced', 
                                                                   np.unique(y_bins), 
                                                                   y_bins)
    class_dict = dict(zip(np.unique(y_bins), class_weight))
    return np.vectorize(class_dict.get)(y_bins)

In [ ]:
X = np.array(images)

# X_train, X_test, y_train, y_test, y_train_weights, y_test_weights = sklearn.model_selection.train_test_split(
#                                                                             X, y, y_weights, 
#                                                                             test_size=0.2, random_state=1)
# X_train, X_val, y_train, y_val, y_train_weights, y_val_weights = sklearn.model_selection.train_test_split(
#                                                                     X_train, y_train, y_train_weights, 
#                                                                     test_size=0.25, random_state=1)

In [ ]:
y = np.array(measurements)
y_weights = bin_weights(y)

#Check whether contains nan
print('y contains nan: {}'.format(np.isnan(np.min(y))))
print('y_weights contains nan: {}'.format(np.isnan(np.min(y_weights))))

In [ ]:
np.unique(y_weights)

In [ ]:
plt.hist(y, bins=10)

Save files to directory so that we can read them with `flow_from_directory`. Shoving them into an array causes memory errors.

### Image Generator
* Images are flipped in order to reduce bias towards driving on slightly towards one side of the road.
* ZCA Whitening is applied
* Image generator probably won't work without classes

In [ ]:
# img_width=320
# img_height=160
# batch_size=32
# datagen = keras.preprocessing.image.ImageDataGenerator(
#             featurewise_center=True,
# #             featurewise_std_normalization=False,
#             zca_whitening=True,
#             horizontal_flip=True)
# datagen.flow_from_directory('./driving_data/train/',
#                             target_size=(img_width, img_height),
#                             batch_size=batch_size,
#                             class_mode=None, 
#                             classes=None)

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Conv2D
from keras.layers import MaxPooling2D, Cropping2D

model = Sequential()
model.add(Cropping2D(cropping=((20,8), (0,0)), input_shape=(64,128,3)))
model.add(Lambda(lambda x: x/255.0 - 0.5))
model.add(Conv2D(24, (5,5), activation='elu', padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(36, (5,5), activation='elu', padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(48, (5,5), activation='elu', padding='same'))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3,3), activation='elu', padding='same'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(100, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(1))
model.compile(loss='mse', optimizer=Adam(lr=0.0001))
history_object = model.fit(X, y, validation_split=0.2, 
          sample_weight=y_weights, shuffle=True, verbose=2)
model.save('model{0}.h5'.format(np.datetime_as_string(np.datetime64('now'))))

In [ ]:
model.summary()

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Conv2D
from keras.layers import MaxPooling2D, Cropping2D, Dropout

model2 = Sequential()
model2.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))
model2.add(Lambda(lambda x: x/255.0 - 0.5))
model2.add(Conv2D(36, (5,5), activation='elu', padding='same'))
model2.add(MaxPooling2D())
model2.add(Conv2D(48, (3,3), activation='elu', padding='same'))
model2.add(MaxPooling2D())
model2.add(Conv2D(64, (3,3), activation='elu', padding='same'))
model2.add(MaxPooling2D())
model2.add(Flatten())
model2.add(Dense(512, activation='elu'))
model2.add(Dropout(0.5))
model2.add(Dense(128, activation='elu'))
model2.add(Dropout(0.5))
model2.add(Dense(32, activation='elu'))
model2.add(Dense(1))
model2.compile(loss='mse', optimizer=Adam(lr=0.0001))
history_object2 = model2.fit(X, y, validation_split=0.2, 
                             sample_weight=y_weights, shuffle=True, 
                             epochs=25, verbose=2)
model2.save('model{0}.h5'.format(np.datetime_as_string(np.datetime64('now'))))

In [ ]:
model2.summary()

In [ ]:
# center = mpimg.imread(log['CenterImg'][0])
# left = mpimg.imread(log['LeftImg'][0])
# right = mpimg.imread(log['RightImg'][0])
fig = plt.figure(figsize=(12,3))
plt.suptitle('Car Images')
axl = plt.subplot(1,3,1)
plt.imshow(left)
axc = plt.subplot(1,3,2)
plt.imshow(center)
axr = plt.subplot(1,3,3)
plt.imshow(right)
plt.show()